# Build a Versatile LLM Agent with LangGraph

Large Language Models (LLMs) have revolutionized how we build different AI applications. AI Agents can handle tasks that require decision-making, real-world interactions, and autonomy. While there are many frameworks available, LangGraph stands out for its ability to create sophisticated agents with multiple capabilities. In this notebook, we'll explore how to build a practical LLM agent for an educational company, "Best Courses," demonstrating three distinct use cases that showcase the versatility of LangGraph.

This bot will handle three types of interactions:
1. General company information (character-based responses)
2. Course-specific queries (structured data handling)
3. Enrollment FAQ assistance (unstructured data processing)

These scenarios are thoughtfully selected to represent common real-world applications and showcase various aspects of agent development.

In [ ]:
# install libraries
!pip install langchain langchain-community langchain-experimental langchain-openai langgraph pandas tabulate PyMuPDF faiss-cpu fastembed

In [ ]:
# load env variables from your local .env file
from dotenv import load_dotenv
load_dotenv()

### langgraph state

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

### langgraph nodes

### general node

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-4o-2024-11-20",
openai_api_key = os.getenv("OPENROUTER_API_KEY"),
openai_api_base = 'https://openrouter.ai/api/v1',)

def general(state: State):
    general_sys_prompt = """
    You are a chatbot for "Best Courses", you will only answer questions about the description below regarding "Best Courses". 

    Welcome to Best Courses - Where Excellence Meets Education
Best Courses is a premier educational institute dedicated to transforming aspirations into achievements. Established in 2010, we have consistently maintained our position as a leading provider of quality education and professional development programs. Our institute stands as a beacon of academic excellence, offering a diverse range of courses designed to meet the evolving demands of today's competitive world.
At Best Courses, we believe in the power of education to change lives. Our state-of-the-art facilities, coupled with an experienced faculty of industry experts and academicians, create an optimal learning environment that nurtures talent and fosters growth. We pride ourselves on maintaining small class sizes to ensure personalized attention and maximum interaction between students and instructors.
Our Offerings:
•	Professional certification courses
•	Industry-specific skill development programs
•	Language proficiency classes
•	Career counseling and placement assistance
•	Soft skills training
•	Technical workshops
•	Online and hybrid learning options
What Sets Us Apart:
1.	Industry-Aligned Curriculum: Our courses are regularly updated to reflect current industry trends and requirements.
2.	Practical Approach: We emphasize hands-on learning through real-world projects and case studies.
3.	Expert Faculty: Our instructors bring years of professional experience and academic excellence to the classroom.
4.	Modern Infrastructure: Well-equipped classrooms, computer labs, and libraries provide all necessary resources for effective learning.
5.	Flexible Learning Options: We offer multiple batches, weekend classes, and online courses to accommodate various schedules.
Success Track Record:
•	Over 50,000 students trained
•	90% placement rate
•	1000+ corporate tie-ups
•	Alumni working in leading organizations worldwide
Our Vision:
To be the most trusted name in education, creating future-ready professionals who excel in their chosen fields and contribute meaningfully to society.
Our Mission:
To provide high-quality, accessible education that empowers individuals with knowledge, skills, and confidence to achieve their professional goals.
Whether you're a fresh graduate looking to enhance your employability, a professional seeking to upgrade your skills, or someone pursuing a career change, Best Courses offers the perfect platform to realize your dreams. Join us on your journey to success, and experience education that truly makes a difference.
Transform your future with Best Courses - Where Learning Leads to Success!

"""
    return {"messages": [llm.invoke([SystemMessage(general_sys_prompt)]+state["messages"])]}

### structured data driven answering node

In [ ]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

df = pd.read_csv("./courses.csv")
agent = create_pandas_dataframe_agent(llm, df, verbose=False, allow_dangerous_code=True, agent_executor_kwargs={"handle_parsing_errors": True})

def courses(state: State):
    ret = agent.invoke(state["messages"][-1])
    return { "messages": ret["output"] }

### unstructured data RAG node

In [ ]:
import faiss
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vector_store = FAISS.load_local("./faq", embeddings,  allow_dangerous_deserialization=True)


# if you did not build the local vector storage yet, you can use the below code to build

# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore= InMemoryDocstore(),
#     index_to_docstore_id={}
# )
# loader = PyMuPDFLoader(r".\FAQ.pdf")
# docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)
# _ = vector_store.add_documents(documents=all_splits)
# vector_store.save_local("./faq")



import os
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-4o-2024-11-20",
openai_api_key = os.getenv("OPENROUTER_API_KEY"),
openai_api_base = 'https://openrouter.ai/api/v1',)


prompt_template = PromptTemplate.from_template("""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
""")


class RagState(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: RagState):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: RagState):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(RagState).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
rag_graph = graph_builder.compile()


def faq(state: State):
    ret = rag_graph.invoke({"question": state["messages"][-1].content})
    return {"messages": ret["answer"]}

### combine the nodes and build the graph

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


def route_conversation(
    state: State,
):

    route_sys_prompt = """
    Given the user input message, choose the best route.
    
    1. courses. if the user input message is related to inquiry about the courses information, e.g. Course Name, Title of Award, Awarding Institution,
    Broad Discipline, Course Duration, Mode of Delivery, General Entry Requirement, Course Fee.

    2. faq. if the user input is about course enrollment faq.

    3. general. anything not related to 1 and 2.

    You will only output "courses", "faq" or "general", without any quote.
"""

    response = llm.invoke([SystemMessage(route_sys_prompt), state["messages"][-1]])

    return response.content

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("general", general)
graph_builder.add_node("faq", faq)
graph_builder.add_node("courses", courses)

graph_builder.add_conditional_edges(
    START,
    route_conversation,
    {"general": "general", "faq": "faq", "courses": "courses"},
)

graph_builder.add_edge("general", END)
graph_builder.add_edge("faq", END)
graph_builder.add_edge("courses", END)
graph = graph_builder.compile()

### invoke the agent

In [ ]:
graph.invoke({"messages": ["how many courses can we choose?"]})